Install libraries

In [1]:
!pip install langchain langchain-huggingface chromadb transformers torch

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
    --------------------------------------- 0.3/18.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.3 MB 1.1 MB/s eta 0:00:16
   - -------------------------------------- 0.8/18.3 MB 1.3 MB/s eta 0:00:14
   -- ------------------------------------- 1.3/18.3 MB 1.5 MB/s eta 0:00:12
   --- ------------------------------------ 1.6/18.3 MB 1.6 MB/s eta 0:00:11
   ---- ----------------------------------- 2.1/18.3 MB 1.8 MB/s eta 0:00:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.14.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


2. Import Libraries

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os, getpass


3. Authenticate Hugging Face

In [3]:
# Prompt for your Hugging Face API key if not already set
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")


4. Load and Split CTSE Lecture Notes

In [4]:
# Load your lecture notes PDF file
loader = PyPDFLoader("CTSE_Lecture_Notes.pdf")  # Replace with your file name
documents = loader.load()

# Split into chunks (important for context-aware retrieval)
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(documents)


5. Create Embeddings & Vector Store

In [5]:
# Use Sentence Transformer to convert text into vectors
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Store vectors using Chroma DB
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_langchain_db"
)


C:\Users\bhanu\AppData\Local\Temp\ipykernel_2964\41131232.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

6. Set Up Retriever

In [6]:
# Create a retriever that returns top 4 relevant chunks
retriever = vector_store.as_retriever(search_kwargs={"k": 4})


7. Initialize Flan-T5 Base LLM

In [7]:
# Load Hugging Face's flan-t5-base model for QA generation
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create a text2text generation pipeline
flan_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=flan_pipeline)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
C:\Users\bhanu\AppData\Local\Temp\ipykernel_2964\1702225085.py:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=flan_pipeline)


8. Build Retrieval QA Chain

In [8]:
# Create LangChain QA chain with the retriever and LLM
qa_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)


9. Define Chatbot Function

In [9]:
# A simple chatbot function to ask questions
def chatbot(query: str) -> str:
    return qa_chain.run(query)


10. Test the Chatbot

In [10]:
# Try sample questions from your lecture notes
questions = [
    "What is software engineering?",
    "Explain the Agile methodology.",
    "What are current trends in AI and ML?",
    "Define DevOps in software development."
]

for q in questions:
    print(f"\n🟡 Question: {q}")
    print(f"🟢 Answer: {chatbot(q)}")



🟡 Question: What is software engineering?


C:\Users\bhanu\AppData\Local\Temp\ipykernel_2964\1223906661.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return qa_chain.run(query)
Token indices sequence length is longer than the specified maximum sequence length for this model (681 > 512). Running this sequence through the model will result in indexing errors


🟢 Answer: Software development practice where developers regularly merge their code changes into a central repository, after which automated builds

🟡 Question: Explain the Agile methodology.
🟢 Answer: Agile is a methodology that combines the use of continuous integration and continuous delivery.

🟡 Question: What are current trends in AI and ML?
🟢 Answer: AI attempts to give machine the ability to mimic human decision making/learning capabilities

🟡 Question: Define DevOps in software development.
🟢 Answer: DevOps is the combination of cultural philosophies, practices, and tools that increases
